# Process Offline Database
* Imports anidb tags from https://github.com/manami-project/anime-offline-database

In [1]:
import json
import os

import pandas as pd
from tqdm import tqdm

In [2]:
os.chdir("../../data/raw_data")

In [3]:
json_fn = "https://github.com/manami-project/anime-offline-database/raw/master/anime-offline-database.json"
! wget $json_fn 

--2022-06-21 20:46:33--  https://github.com/manami-project/anime-offline-database/raw/master/anime-offline-database.json
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/manami-project/anime-offline-database/master/anime-offline-database.json [following]
--2022-06-21 20:46:33--  https://raw.githubusercontent.com/manami-project/anime-offline-database/master/anime-offline-database.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44354090 (42M) [text/plain]
Saving to: ‘anime-offline-database.json’

anime-offline-datab 100%[===================>]  42.30M  80.0MB/s    in 0.5s    

2022-06-21 20:46:

In [4]:
with open('anime-offline-database.json', 'r') as f:
    db = json.load(f)

In [5]:
anime = pd.read_csv("anime.csv")

In [6]:
anime.loc[lambda x: x['genres'].isna(), 'genres'] = '[]' # TODO check if this is redundant
anime['tags'] = '[]'

In [7]:
for entry in tqdm(db['data']):
    mal_id = [x for x in entry['sources'] if 'myanimelist' in x]
    if mal_id:
        mal_id = int(mal_id[0].split('/')[-1])
        anime.loc[lambda x: x.anime_id == mal_id, 'tags'] =  str(entry['tags'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32453/32453 [00:01<00:00, 17002.13it/s]


In [8]:
anime.to_csv('../processed_data/anime.csv', index=False)